In [18]:
import pandas as pd


file_path = 'yelp_academic_dataset_business.json'
df_negocios = pd.read_json(file_path, lines=True)


In [ ]:
columns_check = ['categories']
df_negocios_cl = df_negocios.dropna(subset=columns_check)
print(df_negocios_cl.info())


<class 'pandas.core.frame.DataFrame'>
Index: 150243 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150243 non-null  object 
 1   name          150243 non-null  object 
 2   address       150243 non-null  object 
 3   city          150243 non-null  object 
 4   state         150243 non-null  object 
 5   postal_code   150243 non-null  object 
 6   latitude      150243 non-null  float64
 7   longitude     150243 non-null  float64
 8   stars         150243 non-null  float64
 9   review_count  150243 non-null  int64  
 10  is_open       150243 non-null  int64  
 11  attributes    136601 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 17.2+ MB
None


In [5]:
# checar espaços na string
str_cols =  list(df_negocios.select_dtypes(include='object').columns)
str_cols = [col for col in str_cols if col not in ['postal_code', 'latitude', 'longitude', 'stars', 'review_count','is_open']]

for col in str_cols:
    df_negocios[col] = df_negocios[col].str.strip()

print(df_negocios[str_cols].head())

              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address           city state  attributes  \
0           1616 Chapala St, Ste 2  Santa Barbara    CA         NaN   
1  87 Grasso Plaza Shopping Center         Affton    MO         NaN   
2             5255 E Broadway Blvd         Tucson    AZ         NaN   
3                      935 Race St   Philadelphia    PA         NaN   
4                    101 Walnut St     Green Lane    PA         NaN   

                                          categories  hours  
0  Doctors, Traditional Chinese Medicine, Naturop...    NaN  
1  Shipping Centers, Local Services, Notaries, Ma...    NaN  
2  Department Stores, Shopping, Fashion, Home & G...    

In [19]:
#extrair o horário de funcionamento em colunas separadas (pelo dia da semana)
def extract_hours(hours):
    if pd.isnull(hours):
        return pd.Series([None] * 7, index=['hours_Monday', 'hours_Tuesday', 'hours_Wednesday', 'hours_Thursday', 'hours_Friday', 'hours_Saturday', 'hours_Sunday'])
    else:
        return pd.Series([hours.get(day, None) for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']], index=['hours_Monday', 'hours_Tuesday', 'hours_Wednesday', 'hours_Thursday', 'hours_Friday', 'hours_Saturday', 'hours_Sunday'])

df_hours = df_negocios['hours'].apply(extract_hours)
df_negocios = pd.concat([df_negocios, df_hours], axis=1)

df_negocios.drop('hours', axis=1, inplace=True)

In [20]:
#extrair os atributos em colunas separadas
def extract_atributes(atributes):
    if pd.isnull(atributes):
        return pd.Series()
    else:
        return pd.Series(atributes)
    
df_atributes = df_negocios['attributes'].apply(extract_atributes)
df_negocios = pd.concat([df_negocios, df_atributes], axis=1)
df_negocios.drop('attributes', axis=1, inplace=True)


pd.set_option('display.max_columns', None)
print(df_negocios.head())

              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address           city state postal_code  \
0           1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1  87 Grasso Plaza Shopping Center         Affton    MO       63123   
2             5255 E Broadway Blvd         Tucson    AZ       85711   
3                      935 Race St   Philadelphia    PA       19107   
4                    101 Walnut St     Green Lane    PA       18054   

    latitude   longitude  stars  review_count  is_open  \
0  34.426679 -119.711197    5.0             7        0   
1  38.551126  -90.335695    3.0            15        1   
2  32.223236 -110.880452    3.5            22        0   
3  39.9555

In [8]:
columns = list(df_negocios.columns)
print(columns)

['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'categories', 'hours_Monday', 'hours_Tuesday', 'hours_Wednesday', 'hours_Thursday', 'hours_Friday', 'hours_Saturday', 'hours_Sunday']


In [16]:
columns_check = [
    'ByAppointmentOnly', 'BusinessAcceptsCreditCards', 'BikeParking', 
    'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'RestaurantsDelivery', 
    'WiFi', 'WheelchairAccessible', 'HappyHour', 'RestaurantsReservations', 
    'DogsAllowed', 'Alcohol', 'GoodForKids', 'Ambience', 
    'RestaurantsTableService', 'RestaurantsGoodForGroups', 'DriveThru', 
    'NoiseLevel', 'GoodForMeal', 'Smoking', 'Music', 'GoodForDancing', 
    'BestNights', 'HairSpecializesIn', 'Open24Hours', 
    'RestaurantsCounterService', 'AgesAllowed', 'DietaryRestrictions'
]

# Contar valores não nulos nas colunas especificadas
non_null_counts = df_negocios[columns_check].count()

print(non_null_counts)

ByAppointmentOnly              42339
BusinessAcceptsCreditCards    119765
BikeParking                    72638
RestaurantsPriceRange2         85314
RestaurantsTakeOut             59857
RestaurantsDelivery            56282
WiFi                           56914
WheelchairAccessible           28953
HappyHour                      15171
RestaurantsReservations        45247
DogsAllowed                    18284
Alcohol                        43189
GoodForKids                    53375
Ambience                       44279
RestaurantsTableService        19982
RestaurantsGoodForGroups       44170
DriveThru                       7760
NoiseLevel                     37993
GoodForMeal                    29087
Smoking                         4567
Music                           7521
GoodForDancing                  4628
BestNights                      5694
HairSpecializesIn               1065
Open24Hours                       39
RestaurantsCounterService         19
AgesAllowed                      129
D

In [ ]:
# Remover colunas com menos de 40% de valores não nulos
colunas_para_remover = [
    'BYOB', 'BYOBCorkage', 'Corkage', 'BusinessParking', 
    'BusinessAcceptsBitcoin', 'Caters', 'OutdoorSeating', 
    'RestaurantsAttire', 'CoatCheck','AcceptsInsurance', 'HasTV','DriveThru','Smoking','AgesAllowed','DietaryRestrictions','Music','GoodForDancing',
    'BestNights','HairSpecializesIn','Open24Hours','RestaurantsCounterService'
]

df_negocios = df_negocios.drop(columns=colunas_para_remover)

pd.set_option('display.max_columns', None)
print(df_negocios.head())


              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address           city state postal_code  \
0           1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1  87 Grasso Plaza Shopping Center         Affton    MO       63123   
2             5255 E Broadway Blvd         Tucson    AZ       85711   
3                      935 Race St   Philadelphia    PA       19107   
4                    101 Walnut St     Green Lane    PA       18054   

    latitude   longitude  stars  review_count  is_open  \
0  34.426679 -119.711197    5.0             7        0   
1  38.551126  -90.335695    3.0            15        1   
2  32.223236 -110.880452    3.5            22        0   
3  39.9555

In [25]:
# Preencher valores faltantes com 2 
df_negocios['RestaurantsPriceRange2'] = df_negocios['RestaurantsPriceRange2'].fillna(2)

# Preencher valores faltantes com False
columns_to_fill_false = df_negocios.columns.difference(['RestaurantsPriceRange2'])
df_negocios[columns_to_fill_false] = df_negocios[columns_to_fill_false].fillna(False)

pd.set_option('display.max_columns', None)
print(df_negocios.head())

              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address           city state postal_code  \
0           1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1  87 Grasso Plaza Shopping Center         Affton    MO       63123   
2             5255 E Broadway Blvd         Tucson    AZ       85711   
3                      935 Race St   Philadelphia    PA       19107   
4                    101 Walnut St     Green Lane    PA       18054   

    latitude   longitude  stars  review_count  is_open  \
0  34.426679 -119.711197    5.0             7        0   
1  38.551126  -90.335695    3.0            15        1   
2  32.223236 -110.880452    3.5            22        0   
3  39.9555

In [27]:
#corrigindo a formatação
df_negocios['WiFi'] = df_negocios['WiFi'].str.replace("u'", "").str.replace("'", "")
df_negocios['Alcohol'] = df_negocios['Alcohol'].str.replace("u'", "").str.replace("'", "")

print(df_negocios.head())

              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address           city state postal_code  \
0           1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1  87 Grasso Plaza Shopping Center         Affton    MO       63123   
2             5255 E Broadway Blvd         Tucson    AZ       85711   
3                      935 Race St   Philadelphia    PA       19107   
4                    101 Walnut St     Green Lane    PA       18054   

    latitude   longitude  stars  review_count  is_open  \
0  34.426679 -119.711197    5.0             7        0   
1  38.551126  -90.335695    3.0            15        1   
2  32.223236 -110.880452    3.5            22        0   
3  39.9555

In [ ]:
non_null_counts = df_negocios.count()
print(non_null_counts)

NameError: name 'df_negocios' is not defined

In [ ]:
df_negocios_cl.head()

In [ ]:
#checar latitude e longitude fora do esperado
df_negocios_cl = df_negocios_cl[(df_negocios_cl['latitude'].between(-90, 90)) & (df_negocios_cl['longitude'].between(-180, 180))]

In [ ]:
# checar duplicados
df_negocios_cl = df_negocios_cl.drop_duplicates(subset=['business_id'], keep='first')

In [ ]:
# ver o percentual dos dados excluidos
excluidos = (len(df_negocios) - len(df_negocios_cl)) / len(df_negocios) * 100
print(excluidos)

In [ ]:
# checar dados faltantes
print(df_negocios_cl.isnull().sum())


In [ ]:
df_negocios_cl.head()

In [ ]:
df_negocios_cl.info()

NameError: name 'df_negocios_cl' is not defined

In [ ]:
# normalizar textos
df_negocios_cl['categories'] = df_negocios_cl['categories'].str.lower().str.strip()
df_negocios_cl['city'] = df_negocios_cl['city'].str.lower().str.strip()



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x=df_negocios_cl['review_count'])
plt.show()

sns.histplot(df_negocios_cl['stars'], bins=10, kde=True)
plt.show()


In [ ]:
import folium
from folium.plugins import MarkerCluster


# Criar um mapa básico com pontos de latitude/longitude
mapa = folium.Map(location=[df_negocios_cl['latitude'].mean(), df_negocios_cl['longitude'].mean()], zoom_start=10)

marker_cluster = MarkerCluster().add_to(mapa)

for _, row in df_negocios_cl.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], popup=row['name']).add_to(marker_cluster)

mapa.save('mapa.html')


In [ ]:
estado_contagem = df_negocios_cl['state'].value_counts()
print(estado_contagem)

state
PA    26317
FL    20974
TN     9619
IN     8856
MO     8848
AZ     7793
LA     7644
NJ     6620
NV     5844
AB     4111
CA     3951
ID     3600
DE     1761
IL     1669
TX        2
NC        1
WA        1
CO        1
MT        1
MI        1
HI        1
SD        1
VI        1
VT        1
Name: count, dtype: int64


In [ ]:
file_path = 'yelp_academic_dataset_tip.json'
df_tip = pd.read_json(file_path, lines=True)

df_tip.head()

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [ ]:
columns_tip = list(df_tip.columns)
print(columns_tip)

In [ ]:
df_tip.info()

In [ ]:
df_tip.describe()

In [ ]:
#Filtra os negócios que possuem mais de 100 reviews
contagem_reviews = df_tip['business_id'].value_counts()
ids_mais_reviews = contagem_reviews[contagem_reviews > 100].index

df_tip_mais_reviews = df_tip[df_tip['business_id'].isin(ids_mais_reviews)]
df_tip_mais_reviews.head()

,user_id,business_id,text,date,compliment_count
8,VL12EhEdT4OWqGq0nIqkzw,xODBZmX4EmlVvbqtKN7YKg,Tacos,2012-07-27 01:48:24,0
17,kjFgyrCvmVVGSlgWzRXILw,ReX09lhufLTAx19krkltDA,Got the grilled chicken entree. It was delicio...,2013-06-10 20:18:41,0
22,phKNHTaokisQkZ7qQbxryA,pWuoHHu0LuVrD0HjadDygg,Crepes please!!!,2010-11-21 20:52:49,0
28,CozGguiwx7JEVk7cAwdvpA,ghsaIqQ1xY3cWtWWJ4QW5w,Icky on tap,2010-12-27 20:38:49,0
30,jDThlALkraoQLLBYHqY7FQ,9DJhhBqQSu-gTBwaqdhgpQ,Come early for the best service as they fill u...,2016-01-16 19:13:57,0
